In [4]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np


In [5]:
PROBLEM_SIZE = 15
NUM_SETS = 20
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])


In [7]:
def goal_check(state):                                  
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def h(state):
    already_covered = reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ) 
    
    if np.all(already_covered):     # no additional elements needed, program dies otherwise
        return 0
    
    not_covered = PROBLEM_SIZE - sum(already_covered)   # check which elements are not covered yet, just a number
    
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken_sets = 1
    while sum(candidates[:taken_sets]) < not_covered:
        taken_sets += 1
    return taken_sets   # returns the number of sets that are at least necessary to cover the universe
    

def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    

In [8]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

In [10]:
frontier = PriorityQueue()
#frontier = SimpleQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((len(new_state.taken) + h(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

print("solution: ", [SETS[i] for i in current_state.taken])

Solved in 930 steps (5 tiles)
solution:  [array([False, False, False, False, False,  True, False, False,  True,
       False, False,  True,  True, False, False]), array([ True, False, False, False,  True,  True, False,  True,  True,
       False,  True, False, False, False,  True]), array([False, False,  True, False,  True, False,  True, False, False,
       False, False, False, False, False, False]), array([ True,  True, False,  True, False,  True, False, False, False,
       False, False, False, False,  True, False]), array([False,  True, False, False,  True, False, False, False, False,
        True, False, False, False, False,  True])]


In [11]:
current_state

State(taken={0, 6}, not_taken={1, 2, 3, 4, 5, 7})

In [10]:
SETS[0]

array([False, False,  True,  True,  True])